In [1]:
from scholar import scholar
from habanero import Crossref, cn
from scihub.scihub.scihub import SciHub
import requests
import json
import re

In [3]:
RES_LIMIT = 30

# Google Scholar

In [10]:
# Prepare scholar extractor
querier = scholar.ScholarQuerier()
settings = scholar.ScholarSettings()
settings.set_citation_format(scholar.ScholarSettings.CITFORM_BIBTEX)
querier.apply_settings(settings)

[ INFO]  requesting http://scholar.google.com/scholar_settings?sciifh=1&hl=en&as_sdt=0,5
[ INFO]  requesting http://scholar.google.com/scholar_setprefs?q=&scisig=AAGBfm0AAAAAWmZQfpflxnWKB9QRSBN2sPKp2dgxY_fc&inststart=0&as_sdt=1,5&as_sdtp=&num=None&scis=yes&scisf=4&hl=en&lang=all&instq=&inst=569367360547434339&save=
[ INFO]  settings applied


True

In [11]:
def submit(author, keyphrase):
    query = scholar.SearchScholarQuery()
    query.set_author(author)
    query.set_phrase(keyphrase)
    query.set_num_page_results(RES_LIMIT)
    querier.send_query(query)
    results = scholar.json(querier)
    return results

In [9]:
author = "fodor"
phrase = "language"

In [14]:
data

[{'authors': ['P Gärdenfors'],
  'cluster_id': '3985340221429114199',
  'excerpt': "IMAGING AND CONDITIONALIZATION 749 lion of Lewis's process which does not rely on the assumption that, for every world W, there is a unique world that is the most similar to W where a given sentence A is true. Secondly, I shall give a characterization of the generalized ",
  'num_citations': 73,
  'num_versions': 5,
  'title': 'Imaging and conditionalization',
  'url': 'https://www.pdcnet.org/jphil/content/jphil_1982_0079_0012_0747_0760',
  'url_citation': 'https://scholar.googleusercontent.com/scholar.bib?q=info:V-Vb79DFTjcJ:scholar.google.com/&output=citation&scisig=AAGBfm0AAAAAWmZQpzsCPauD8obGRvsvTos2HLGaJXrt&scisf=4&ct=citation&cd=0&hl=en',
  'url_citations': 'http://scholar.google.com/scholar?cites=3985340221429114199&as_sdt=2005&sciodt=0,5&hl=en',
  'url_pdf': None,
  'url_versions': 'http://scholar.google.com/scholar?cluster=3985340221429114199&hl=en&as_sdt=0,5',
  'year': '1982'},
 {'authors': [

# Scopus API

In [6]:
url = "http://api.elsevier.com/content/search/scopus"
key = '23096dfc3e1a8d87504a227fb4314c9f'

In [7]:
def scopus_submit(author, phrase):
    query = re.sub("\s+", "\+", phrase)
    if author != "":
        author = "AUTH%28{}%29".format(re.sub("\s+", "\+", author))
        query = "{}+{}".format(author, query)
    view = "STANDARD"
    count = "10"
    r = requests.get("{}?query={}&apiKey={}&view={}&count={}".format(url, query, key, view, count))
    return r.json()

In [10]:
data = scopus_submit(author, phrase)

In [11]:
len(data['search-results']['entry'])

10

In [12]:
data['search-results']['entry'][1]['prism:doi']

'10.1177/0023830916665653'

In [13]:
doi = data['search-results']['entry'][6]['prism:doi']

In [14]:
doi

'10.1371/journal.pone.0170208'

# Get article from Unpaywall

In [35]:
# http://unpaywall.org/api/v2

In [55]:
def unpaywall_check(doi):
    r = requests.get("https://api.oadoi.org/v2/{}?email=test@example.com".format(doi))
    response = r.json()
    return response['is_oa']

In [56]:
# https://stackoverflow.com/questions/15644964/python-progress-bar-and-downloads

def unpaywall_download(doi, save="test.pdf"):
    r = requests.get("https://api.oadoi.org/v2/{}?email=test@example.com".format(doi))
    response = r.json()
    for loc in response['oa_locations']:
        if loc['url_for_pdf']:
            dl = requests.get(loc['url_for_pdf'])
            with open(save, 'wb') as output:
                output.write(dl.content)
            return True
        else:
            continue
    return False

In [57]:
unpaywall_check(doi)

True

In [58]:
unpaywall_download(doi)

True

# Get article from Scihub

In [60]:
sh = SciHub()

In [63]:
def scihub_download(doi, save="text2.pdf"):
    try:
        dl = sh.download(doi)
    except Exception as e:
        print(e)
        return False
#     with open(save, 'wb') as output:
#         output.write(dl['pdf'])
    return True

In [62]:
scihub_download(doi)

True

# Get citations from Crossref

In [15]:
cr = Crossref()

In [16]:
print(cn.content_negotiation(ids=doi, format="bibentry"))

@article{Kleiman_2017,
	doi = {10.1371/journal.pone.0170208},
	url = {https://doi.org/10.1371%2Fjournal.pone.0170208},
	year = 2017,
	month = {jan},
	publisher = {Public Library of Science ({PLoS})},
	volume = {12},
	number = {1},
	pages = {e0170208},
	author = {Susan C. Kleiman and Emily C. Bulik-Sullivan and Elaine M. Glenny and Stephanie C. Zerwas and Eun Young Huh and Matthew C. B. Tsilimigras and Anthony A. Fodor and Cynthia M. Bulik and Ian M. Carroll},
	editor = {Brenda A Wilson},
	title = {The Gut-Brain Axis in Healthy Females: Lack of Significant Association between Microbial Composition and Diversity with Psychiatric Measures},
	journal = {{PLOS} {ONE}}
}


In [1]:
# cn.content_negotiation(ids = '10.1126/science.169.3946.635')
# cn.content_negotiation(ids = '10.1126/science.169.3946.635', format = "citeproc-json")
# cn.content_negotiation(ids = "10.1126/science.169.3946.635", format = "rdf-xml")
# cn.content_negotiation(ids = "10.1126/science.169.3946.635", format = "text")
# cn.content_negotiation(ids = "10.1126/science.169.3946.635", format = "text", style = "apa")
# cn.content_negotiation(ids = "10.1126/science.169.3946.635", format = "bibentry")

In [35]:
x = cr.works(query = "ecology", offset=100)

In [41]:
x['message']['items'][0]

{'DOI': '10.1007/978-94-017-1899-8_7',
 'ISBN': ['9789048142989', '9789401718998'],
 'ISSN': ['0926-9754'],
 'URL': 'http://dx.doi.org/10.1007/978-94-017-1899-8_7',
 'author': [{'affiliation': [], 'family': 'Ruth', 'given': 'Matthias'}],
 'container-title': ['Ecology, Economy & Environment',
  'Integrating Economics, Ecology and Thermodynamics'],
 'content-domain': {'crossmark-restriction': False, 'domain': []},
 'created': {'date-parts': [[2013, 3, 8]],
  'date-time': '2013-03-08T13:18:19Z',
  'timestamp': 1362748699000},
 'deposited': {'date-parts': [[2013, 3, 8]],
  'date-time': '2013-03-08T13:18:27Z',
  'timestamp': 1362748707000},
 'indexed': {'date-parts': [[2017, 10, 26]],
  'date-time': '2017-10-26T05:09:26Z',
  'timestamp': 1508994566458},
 'is-referenced-by-count': 0,
 'issn-type': [{'type': 'print', 'value': '0926-9754'}],
 'issued': {'date-parts': [[1993]]},
 'link': [{'URL': 'http://link.springer.com/content/pdf/10.1007/978-94-017-1899-8_7',
   'content-type': 'unspecified